In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

possible_paths = [
    "data_scaled/insurance_cleaned_knn_minmax_scaled.csv",
    "data_scaled/insurance_cleaned_knn_standard_scaled.csv",
    "data_scaled/insurance_cleaned_mean_median_minmax_scaled.csv",
    "data_scaled/insurance_cleaned_mean_median_standard_scaled.csv",
    "data_scaled/insurance_cleaned_regression_minmax_scaled.csv",
    "data_scaled/insurance_cleaned_regression_standard_scaled.csv",
    "data_scaled/insurance_cleaned_sklearn_impute_minmax_scaled.csv",
    "data_scaled/insurance_cleaned_sklearn_impute_standard_scaled.csv"
]

df = None
for p in possible_paths:
    if os.path.exists(p):
        df = pd.read_csv(p)
        print(f"Loaded data from: {p}")
        break

target = 'charges'

X = df.drop(columns=[target])
y = df[target]

# Proste kodowanie zmiennych kategorycznych
X = pd.get_dummies(X, drop_first=True)

# %%
# Wykonaj podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print("Podział na train/test wykonany:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

# %%
# Zapisz splity do katalogu data_converted/ (tworzymy katalog jeśli nie istnieje)
out_dir = 'data_splits'
os.makedirs(out_dir, exist_ok=True)

X_train.to_csv(os.path.join(out_dir, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(out_dir, 'X_test.csv'), index=False)

y_train.to_csv(os.path.join(out_dir, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(out_dir, 'y_test.csv'), index=False)

print(f"Zapisano splity w: {out_dir}/")


Loaded data from: data_scaled/insurance_cleaned_knn_minmax_scaled.csv
Podział na train/test wykonany:
X_train: (1068, 8), X_test: (267, 8)
y_train: (1068,), y_test: (267,)
Zapisano splity w: data_splits/


In [5]:
import warnings
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import joblib

warnings.filterwarnings("ignore")

out_dir = 'data_splits'
if 'X_train' not in globals() or 'X_test' not in globals():
    required = [os.path.join(out_dir, n) for n in ('X_train.csv','X_test.csv','y_train.csv','y_test.csv')]
    if not all(os.path.exists(p) for p in required):
        raise FileNotFoundError(f"Brakuje plików w: {out_dir}/ — oczekiwane: {required}")
    X_train = pd.read_csv(required[0])
    X_test  = pd.read_csv(required[1])
    y_train = pd.read_csv(required[2]).squeeze()
    y_test  = pd.read_csv(required[3]).squeeze()

# Przygotuj katalog do zapisu modeli
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

# Definicja modeli scikit-learn
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Dodaj modele zewnętrzne jeśli dostępne
try:
    from xgboost import XGBRegressor
    models["XGBoost"] = XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)
except Exception:
    print("XGBoost nie jest zainstalowany — pomijam.")

try:
    from lightgbm import LGBMRegressor
    models["LightGBM"] = LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1)
except Exception:
    print("LightGBM nie jest zainstalowany — pomijam.")

# Trening, ewaluacja i zapis
results = {}
for name, model in models.items():
    try:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        r2 = r2_score(y_test, preds)
        results[name] = {"rmse": float(rmse), "r2": float(r2)}
        joblib.dump(model, os.path.join(models_dir, f"{name}.joblib"))
        print(f"{name}: RMSE={rmse:.4f}, R2={r2:.4f}  -> saved to {models_dir}/{name}.joblib")
    except Exception as e:
        print(f"Nie udało się wytrenować/zapisać modelu {name}: {e}")

# Podsumowanie wyników
print("\nPodsumowanie wyników:")
for k, v in results.items():
    print(f"{k:15s}  RMSE={v['rmse']:.4f}  R2={v['r2']:.4f}")
# %%

LinearRegression: RMSE=7010.6536, R2=0.6884  -> saved to models/LinearRegression.joblib
Ridge: RMSE=7020.6311, R2=0.6875  -> saved to models/Ridge.joblib
RandomForest: RMSE=6408.6460, R2=0.7396  -> saved to models/RandomForest.joblib
GradientBoosting: RMSE=6262.3726, R2=0.7514  -> saved to models/GradientBoosting.joblib
XGBoost: RMSE=6961.5769, R2=0.6928  -> saved to models/XGBoost.joblib
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000283 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 324
[LightGBM] [Info] Number of data points in the train set: 1068, number of used features: 8
[LightGBM] [Info] Start training from score 12693.808952
LightGBM: RMSE=6564.9821, R2=0.7268  -> saved to models/LightGBM.joblib

Podsumowanie wyników:
LinearRegression  RMSE=7010.6536  R2=0.6884
Ridge            RMSE=7020.6311  R2=0.6875
RandomForest     RM

In [6]:
from sklearn.ensemble import VotingRegressor, StackingRegressor

estimators = [
    ("rf", RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ("gbr", GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ("lr", LinearRegression()),
    ("ridge", Ridge(alpha=1.0, random_state=42)),
    ("xgb", XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)),
    ("lgbm", LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1))
]

# VotingRegressor
try:
    voting = VotingRegressor(estimators=estimators, n_jobs=-1)
    voting.fit(X_train, y_train)
    preds_v = voting.predict(X_test)
    rmse_v = np.sqrt(mean_squared_error(y_test, preds_v))
    r2_v = r2_score(y_test, preds_v)
    results["VotingRegressor"] = {"rmse": float(rmse_v), "r2": float(r2_v)}
    joblib.dump(voting, os.path.join(models_dir, "VotingRegressor.joblib"))
    print(f"VotingRegressor: RMSE={rmse_v:.4f}, R2={r2_v:.4f} -> saved to {models_dir}/VotingRegressor.joblib")
except Exception as e:
    print(f"Nie udało się wytrenować/zapisać VotingRegressor: {e}")

# StackingRegressor
try:
    stacking = StackingRegressor(estimators=estimators, final_estimator=Ridge(random_state=42), n_jobs=-1, passthrough=False)
    stacking.fit(X_train, y_train)
    preds_s = stacking.predict(X_test)
    rmse_s = np.sqrt(mean_squared_error(y_test, preds_s))
    r2_s = r2_score(y_test, preds_s)
    results["StackingRegressor"] = {"rmse": float(rmse_s), "r2": float(r2_s)}
    joblib.dump(stacking, os.path.join(models_dir, "StackingRegressor.joblib"))
    print(f"StackingRegressor: RMSE={rmse_s:.4f}, R2={r2_s:.4f} -> saved to {models_dir}/StackingRegressor.joblib")
except Exception as e:
    print(f"Nie udało się wytrenować/zapisać StackingRegressor: {e}")

# Podsumowanie wyników
print("\nPodsumowanie wyników ensemble:")
for k, v in results.items():
    print(f"{k:20s}  RMSE={v['rmse']:.4f}  R2={v['r2']:.4f}")

VotingRegressor: RMSE=6350.8568, R2=0.7443 -> saved to models/VotingRegressor.joblib
StackingRegressor: RMSE=6297.8466, R2=0.7486 -> saved to models/StackingRegressor.joblib

Podsumowanie wyników ensemble:
LinearRegression      RMSE=7010.6536  R2=0.6884
Ridge                 RMSE=7020.6311  R2=0.6875
RandomForest          RMSE=6408.6460  R2=0.7396
GradientBoosting      RMSE=6262.3726  R2=0.7514
XGBoost               RMSE=6961.5769  R2=0.6928
LightGBM              RMSE=6564.9821  R2=0.7268
VotingRegressor       RMSE=6350.8568  R2=0.7443
StackingRegressor     RMSE=6297.8466  R2=0.7486


In [7]:
# --- Wczytaj dane testowe ---
out_dir = 'data_splits'
required = [os.path.join(out_dir, n) for n in ('X_test.csv','y_test.csv')]
if not all(os.path.exists(p) for p in required):
    raise FileNotFoundError(f"Brakuje plików w: {out_dir}/ — oczekiwane: {required}")

X_test = pd.read_csv(required[0])
y_test = pd.read_csv(required[1]).squeeze().to_numpy()

# --- Definicje metryk (ręcznie) ---
def mse(y_true, y_pred):
    err = y_true - y_pred
    return np.mean(err**2)

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def median_ae(y_true, y_pred):
    return np.median(np.abs(y_true - y_pred))

def mape(y_true, y_pred):
    # bezdzielnik przy zerach - pomijamy elementy z y_true == 0
    denom = np.where(y_true == 0, np.nan, y_true)
    pct_errors = np.abs((y_true - y_pred) / denom)
    # jeśli wszystkie y_true==0 zwróć nan
    return np.nanmean(pct_errors) * 100.0

def r2_score_manual(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1.0 - ss_res / ss_tot if ss_tot != 0 else np.nan

def explained_variance(y_true, y_pred):
    var_res = np.var(y_true - y_pred)
    var_true = np.var(y_true)
    return 1.0 - var_res / var_true if var_true != 0 else np.nan

# --- Zbierz modele: użyj istniejącej zmiennej 'models' jeśli jest, inaczej załaduj .joblib z katalogu models/ ---
models_dict = {}
if 'models' in globals() and isinstance(models, dict):
    # jeżeli wartości to obiekty modeli
    for name, mod in models.items():
        try:
            # jeśli model nie jest wytrenowany, predict może rzucić; obsłużymy wyjątek później
            models_dict[name] = mod
        except Exception:
            pass
else:
    models_dir = 'models'
    if os.path.exists(models_dir):
        for f in os.listdir(models_dir):
            if f.endswith('.joblib') or f.endswith('.pkl'):
                name = os.path.splitext(f)[0]
                try:
                    models_dict[name] = joblib.load(os.path.join(models_dir, f))
                except Exception as e:
                    print(f"Nie udało się załadować modelu {f}: {e}")

if not models_dict:
    raise RuntimeError("Brak dostępnych modeli do oceny (brak zmiennej `models` i brak plików w `models/`).")

# --- Ewaluacja ---
rows = []
for name, model in models_dict.items():
    try:
        preds = model.predict(X_test)
        preds = np.array(preds).squeeze()
        y_true = np.array(y_test).squeeze()
        # ujednolicenie długości
        if preds.shape != y_true.shape:
            print(f"Model {name} - niespójne kształty pred/y: {preds.shape} vs {y_true.shape}, pomijam.")
            continue
        vals = {
            "model": name,
            "n_samples": int(y_true.size),
            "RMSE": float(rmse(y_true, preds)),
            "MAE": float(mae(y_true, preds)),
            "Median_AE": float(median_ae(y_true, preds)),
            "MAPE_%": float(mape(y_true, preds)) if not np.isnan(mape(y_true, preds)) else np.nan,
            "R2": float(r2_score_manual(y_true, preds)),
            "ExplainedVar": float(explained_variance(y_true, preds))
        }
        rows.append(vals)
    except Exception as e:
        print(f"Nie udało się ocenić modelu {name}: {e}")

# --- Wyniki: DataFrame, wydruk i zapis ---
metrics_df = pd.DataFrame(rows).sort_values(by='RMSE')
print(metrics_df.to_string(index=False))

os.makedirs('metrics', exist_ok=True)
metrics_df.to_csv(os.path.join('metrics', 'metrics.csv'), index=False)
print(f"\nZapisano metryki do: metrics/metrics.csv")

           model  n_samples        RMSE         MAE   Median_AE    MAPE_%       R2  ExplainedVar
GradientBoosting        267 6262.372562 4041.655409 2453.214910 47.651951 0.751389      0.751769
    RandomForest        267 6408.646028 4192.134573 2461.138272 47.204999 0.739640      0.739649
        LightGBM        267 6564.982083 4297.189138 2816.928451 49.104254 0.726782      0.728068
         XGBoost        267 6961.576888 4570.305830 2807.239758 53.659345 0.692774      0.692856
LinearRegression        267 7010.653603 4943.082393 3129.907520 50.386786 0.688427      0.689017
           Ridge        267 7020.631104 4950.449881 3099.069207 50.642668 0.687540      0.688158

Zapisano metryki do: metrics/metrics.csv


In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set(style="whitegrid", rc={"figure.figsize": (8, 5)})

# --- ustawienia ścieżek ---
metrics_fp = os.path.join('metrics', 'metrics.csv')
models_dir = 'models'
reports_dir = 'reports'
os.makedirs(reports_dir, exist_ok=True)

# --- wczytaj testowe dane ---
out_dir = 'data_splits'
X_test = pd.read_csv(os.path.join(out_dir, 'X_test.csv'))
y_test = pd.read_csv(os.path.join(out_dir, 'y_test.csv')).squeeze().to_numpy()

# --- wczytaj tabelę metryk jeśli istnieje ---
if os.path.exists(metrics_fp):
    metrics_df = pd.read_csv(metrics_fp)
else:
    metrics_df = None

# --- załaduj modele (preferuj zmienną `models` jeśli jest) ---
models_dict = {}
if 'models' in globals() and isinstance(models, dict) and models:
    models_dict = models.copy()
else:
    if os.path.exists(models_dir):
        for f in os.listdir(models_dir):
            if f.endswith('.joblib') or f.endswith('.pkl'):
                name = os.path.splitext(f)[0]
                try:
                    models_dict[name] = joblib.load(os.path.join(models_dir, f))
                except Exception as e:
                    print(f"Nie udało się załadować {f}: {e}")

if not models_dict:
    raise RuntimeError("Brak modeli w `models` ani w katalogu `models/`.")

# --- oblicz dodatkowe statystyki i reszty ---
rows = []
preds_all = {}
for name, model in models_dict.items():
    try:
        preds = np.array(model.predict(X_test)).squeeze()
    except Exception as e:
        print(f"Predict error {name}: {e}")
        continue
    preds_all[name] = preds
    err = y_test - preds
    abs_err = np.abs(err)
    mse = np.mean(err**2)
    rmse = np.sqrt(mse)
    mae = np.mean(abs_err)
    median_ae = np.median(abs_err)
    # MAPE: pomijamy y_true == 0
    denom = np.where(y_test == 0, np.nan, y_test)
    mape = np.nanmean(np.abs((y_test - preds) / denom)) * 100.0
    ss_res = np.sum(err**2)
    ss_tot = np.sum((y_test - np.mean(y_test))**2)
    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else np.nan
    explained = 1 - np.var(err) / np.var(y_test) if np.var(y_test) != 0 else np.nan
    rows.append({
        "model": name,
        "n_samples": int(y_test.size),
        "RMSE": float(rmse),
        "MAE": float(mae),
        "Median_AE": float(median_ae),
        "MAPE_%": float(mape) if not np.isnan(mape) else np.nan,
        "R2": float(r2),
        "ExplainedVar": float(explained)
    })

comp_df = pd.DataFrame(rows).sort_values(by='RMSE').reset_index(drop=True)
print("\nTabela porównawcza (posortowane po RMSE):")
print(comp_df.to_string(index=False))

# zapisz uaktualnione metryki
os.makedirs('metrics', exist_ok=True)
comp_df.to_csv(os.path.join('metrics', 'metrics_full.csv'), index=False)

# --- wykres 1: słupek RMSE i R2 ---
fig, ax1 = plt.subplots(figsize=(10,5))
sns.barplot(x='RMSE', y='model', data=comp_df, palette='mako', ax=ax1)
ax1.set_title('RMSE (niższe = lepsze)')
plt.tight_layout()
plt.savefig(os.path.join(reports_dir, 'rmse_bar.png'))
plt.close(fig)

fig, ax = plt.subplots(figsize=(10,5))
sns.barplot(x='R2', y='model', data=comp_df, palette='crest', ax=ax)
ax.set_title('R2 (wyższe = lepsze)')
plt.tight_layout()
plt.savefig(os.path.join(reports_dir, 'r2_bar.png'))
plt.close(fig)

# --- wybierz top2 do dalszych analiz ---
top_models = comp_df['model'].tolist()[:2]
print(f"\nTop2 modele do dalszej analizy: {top_models}")

# --- wykres 2: scatter y_true vs y_pred dla top3 (jeśli jest) ---
top3 = comp_df['model'].tolist()[:3]
fig, axes = plt.subplots(1, len(top3), figsize=(5*len(top3),5))
if len(top3) == 1:
    axes = [axes]
for ax, name in zip(axes, top3):
    preds = preds_all[name]
    sns.scatterplot(x=y_test, y=preds, alpha=0.6, ax=ax)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    ax.set_title(f"{name}\nRMSE={comp_df.loc[comp_df.model==name,'RMSE'].values[0]:.0f}")
    ax.set_xlabel("y_true")
    ax.set_ylabel("y_pred")
plt.tight_layout()
plt.savefig(os.path.join(reports_dir, 'ytrue_ypred_top3.png'))
plt.close(fig)

# --- wykres 3: histogram reszt i wykres reszt vs pred dla najlepszego modelu ---
best = top_models[0]
best_preds = preds_all[best]
resid = y_test - best_preds

fig, axs = plt.subplots(1,2, figsize=(12,5))
sns.histplot(resid, bins=30, kde=True, ax=axs[0])
axs[0].set_title(f"Histogram reszt: {best}")
sns.scatterplot(x=best_preds, y=resid, alpha=0.6, ax=axs[1])
axs[1].axhline(0, color='r', linestyle='--')
axs[1].set_title(f"Reszty vs przewidywane: {best}")
axs[1].set_xlabel("y_pred")
axs[1].set_ylabel("resid")
plt.tight_layout()
plt.savefig(os.path.join(reports_dir, 'resid_best.png'))
plt.close(fig)

# --- wykres 4: boxplot błędów bezwzględnych wszystkich modeli ---
ae_df = pd.DataFrame({name: np.abs(preds_all[name] - y_test) for name in preds_all.keys()})
ae_melt = ae_df.melt(var_name='model', value_name='abs_error')
plt.figure(figsize=(10,6))
order = comp_df['model'].tolist()
sns.boxplot(x='abs_error', y='model', data=ae_melt, order=order, palette='vlag')
plt.title('Boxplot błędów bezwzględnych (porównanie modeli)')
plt.xlabel('Absolute error')
plt.tight_layout()
plt.savefig(os.path.join(reports_dir, 'abs_error_boxplot.png'))
plt.close()

# --- test parowy (ttest_rel) między top2 modeli na absolutnych błędach ---
if len(top_models) >= 2:
    a = np.abs(preds_all[top_models[0]] - y_test)
    b = np.abs(preds_all[top_models[1]] - y_test)
    # sprawdź zgodność długości:
    if a.shape == b.shape:
        tstat, pval = stats.ttest_rel(a, b, nan_policy='omit')
        mean_diff = np.mean(a - b)
        print(f"\nPaired t-test (abs errors) {top_models[0]} vs {top_models[1]}:")
        print(f" t = {tstat:.3f}, p = {pval:.3e}, mean(abs_err_diff) = {mean_diff:.3f} ({'+' if mean_diff>0 else '-'})")
        with open(os.path.join(reports_dir, 'paired_ttest.txt'), 'w') as f:
            f.write(f"Paired t-test {top_models[0]} vs {top_models[1]}:\n t = {tstat:.6f}\n p = {pval:.6e}\n mean_abs_err_diff = {mean_diff:.6f}\n")
    else:
        print("Top2 modele mają różne kształty predykcji — nie można przeprowadzić t-testu.")
else:
    print("Za mało modeli do testu parowego.")

print(f"\nWykresy i pliki raportów zapisano w: {reports_dir}/  (pliki: rmse_bar.png, r2_bar.png, ytrue_ypred_top3.png, resid_best.png, abs_error_boxplot.png, paired_ttest.txt jeśli dostępny)")


Tabela porównawcza (posortowane po RMSE):
           model  n_samples        RMSE         MAE   Median_AE    MAPE_%       R2  ExplainedVar
GradientBoosting        267 6262.372562 4041.655409 2453.214910 47.651951 0.751389      0.751769
    RandomForest        267 6408.646028 4192.134573 2461.138272 47.204999 0.739640      0.739649
        LightGBM        267 6564.982083 4297.189138 2816.928451 49.104254 0.726782      0.728068
         XGBoost        267 6961.576888 4570.305830 2807.239758 53.659345 0.692774      0.692856
LinearRegression        267 7010.653603 4943.082393 3129.907520 50.386786 0.688427      0.689017
           Ridge        267 7020.631104 4950.449881 3099.069207 50.642668 0.687540      0.688158

Top2 modele do dalszej analizy: ['GradientBoosting', 'RandomForest']

Paired t-test (abs errors) GradientBoosting vs RandomForest:
 t = -1.249, p = 2.129e-01, mean(abs_err_diff) = -150.479 (-)

Wykresy i pliki raportów zapisano w: reports/  (pliki: rmse_bar.png, r2_bar.png, yt